In [1]:
import torch
import torchvision as tv
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import numpy as np
import pandas as pd
import pathlib
import cv2
import itertools
import random 

from torchvision import datasets
from torch import optim
from scipy.sparse import lil_matrix, csr_matrix
from scipy.optimize import linprog
from matplotlib import pyplot as plt

In [2]:
root = "../../OT-ICNN-bias/data/celeba/"
IMAGE_FOLDER_PATH = root + "Img_folder"
ATTRIBUTES_PATH = root + "list_attr_celeba.csv"

In [3]:
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

In [4]:
celeba = datasets.ImageFolder(IMAGE_FOLDER_PATH)

In [5]:
df = pd.read_csv(ATTRIBUTES_PATH)
df.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [6]:
df.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [7]:
df1 = df[(df["Male"] == -1)].sample(50000)
df1.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
97875,097876.jpg,-1,-1,-1,1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
79742,079743.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
183868,183869.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,1
103338,103339.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,-1,1,-1,-1,1
130185,130186.jpg,-1,-1,-1,-1,-1,-1,1,1,-1,...,-1,-1,-1,-1,-1,1,1,-1,-1,1


In [8]:
df2 = df[(df["Male"] == 1) & (df["No_Beard"] == 1)].sample(40000)
df2 = pd.concat((df2, df[(df["Male"] == 1) & (df["No_Beard"] == -1)].sample(10000)), axis = 0)

df2.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
115267,115268.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
115025,115026.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
129618,129619.jpg,1,-1,1,1,-1,-1,1,1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
134250,134251.jpg,1,-1,1,-1,-1,1,-1,-1,1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
159608,159609.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1


In [9]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

In [10]:
df1.to_csv(root + "/celebA_sample_female.csv", index = False)
df2.to_csv(root + "/celebA_sample_male.csv", index = False)

In [14]:
celebA_male =  df[df["Male"] == 1]
celebA_female =  df[df["Male"] == -1]

celebA_male.reset_index(inplace=True)
celebA_female.reset_index(inplace=True)

celebA_male.to_csv(root + "/celebA_male.csv", index = False)
celebA_female.to_csv(root + "/celebA_female.csv", index = False)

In [15]:
((df["Oval_Face"] == 1) & (df["Male"] == -1)).sum()/len(df[df["Male"] == -1])

0.33002157999407605

In [16]:
((df["Oval_Face"] == 1) & (df["Male"] == 1)).sum()/len(df[df["Male"] == 1])

0.21993509723571072

In [13]:
len(df[df["Male"] == -1])/len(df)

0.5832457218446291